In [90]:
# for reloading the editable module
%load_ext autoreload
%autoreload 2

from pathlib import Path
from dotenv import load_dotenv
import holidays
from datetime import timezone, datetime
import numpy as np
import pandas as pd



from probabilistic_load_forecast.application.services import (
    GetActualLoadData,
    GetERA5DataFromDB,
)



# Import adapters for data access
from probabilistic_load_forecast.adapters.db import (
    EntsoePostgreRepository,
    Era5PostgreRepository,
)

# Import configuration
from probabilistic_load_forecast import config

# Load environment variables

if not load_dotenv(Path("../..") / ".env"):
    raise FileNotFoundError("Could not open the .env file.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
# EXPERIMENT_NAME = "fs_03_load_calendar_weather"
OUTPUT_PATH = Path("../../data/processed")

OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

INCLUDE_WEATHER = True
INCLUDE_FUTURE_WEATHER = True
INCLUDE_CALENDAR = True

In [92]:
# Initialize database repositories
try:
    # Load data repository
    load_repo = EntsoePostgreRepository(config.get_postgre_uri())

    # ERA5 weather data repository
    era5_repo = Era5PostgreRepository(config.get_postgre_uri())

    print("Database repositories initialized successfully")
except Exception as e:
    print(f"Error initializing repositories: {e}")
    print("Make sure your .env file contains the correct database credentials")

Database repositories initialized successfully


In [93]:
# Initialize services
get_load_data = GetActualLoadData(load_repo)
get_era5_data = GetERA5DataFromDB(era5_repo)

print("Services initialized successfully")

Services initialized successfully


In [94]:
start = datetime(2018, 10, 1, 0, 0, tzinfo=timezone.utc)
end = datetime(2025, 10, 10, 0, 0, tzinfo=timezone.utc)

era5_variables = ["t2m", "u10", "v10", "ssrd", "tp"]
era5_data = get_era5_data(
    variables=era5_variables, country_code="AT", start=start, end=end
)
actual_load_ts = get_load_data(start, end)

data_combined = pd.DataFrame()

## Preprocess Timeseries 

In [95]:
# Make sure the timestamps are all sorted

actual_load_ts.data = actual_load_ts.data.sort_index()

era5_data = {
    var: ts.sort_index()
    for var, ts in era5_data.items()
}

actual_load_ts.data.index = (
    actual_load_ts.data.index
    .to_timestamp(how="start") 
    .tz_localize("UTC")
)

In [96]:
if INCLUDE_WEATHER:
    # Convert the total amount measurements per time (ssrd and tp) into a rate
    era5_data = {
        var: (
            ts/3600
            if isinstance(ts.index, pd.PeriodIndex)
            else ts
        )
        for var, ts in era5_data.items()
    }

    # All ts with PeriodIndex are converted to DatetimeIndex
    era5_data = {
        var: (
            ts.set_axis(
                ts.index.to_timestamp(how="start",).tz_localize("UTC")
            )
            if isinstance(ts.index, pd.PeriodIndex)
            else ts
        )
        for var, ts in era5_data.items()
    }

    # FFill to meet the 15min load precision
    era5_ffilled  = {
        var: ts.reindex(actual_load_ts.data.index, method="ffill") for var, ts in era5_data.items()
    }

    # Combine the weather data into a single df
    era5_df = pd.concat(era5_ffilled.values(), axis=1)
    era5_df.columns = era5_ffilled.keys()

    # Compute the windspeed feature
    era5_df["wind_speed"] = np.sqrt(
        era5_df["u10"]**2 + era5_df["v10"]**2
    )

    era5_df = era5_df.drop(columns=["u10", "v10"])

## Merge Era5 and Load Ts

In [97]:
if INCLUDE_WEATHER:
    data_combined = pd.concat([actual_load_ts.data, era5_df], axis=1)
else:
    data_combined = actual_load_ts.data

In [98]:
if INCLUDE_FUTURE_WEATHER:
    data_combined["t2m_future"] = data_combined["t2m"].shift(-96)
    data_combined["ssrd_future"] = data_combined["ssrd"].shift(-96)
    data_combined["tp_future"] = data_combined["tp"].shift(-96)
    data_combined["wind_speed_future"] = data_combined["wind_speed"].shift(-96)

## Calendar Features

In [99]:
if INCLUDE_CALENDAR:
    data_combined["is_weekday"] = data_combined.index.weekday < 5

    # years = range(data_combined.index.min().year, data_combined.index.max().year + 1)
    # at_holidays = holidays.country_holidays("AT", years=years)
    # data_combined["is_holiday"] = data_combined.index.map(lambda x: x in at_holidays)

    idx = data_combined.index

    # Austria public holidays
    years = range(idx.min().year, idx.max().year + 1)
    at_holidays = holidays.country_holidays("AT", years=years)
    data_combined["is_holiday"] = idx.map(lambda ts: ts.date() in at_holidays).astype("int8")

    # Time-of-day (fraction of day
    seconds = (idx.hour * 3600 + idx.minute * 60 + idx.second).to_numpy()
    tod_frac = seconds / 86400.0
    data_combined["sin_tod"] = np.sin(2 * np.pi * tod_frac).astype("float32")
    data_combined["cos_tod"] = np.cos(2 * np.pi * tod_frac).astype("float32")

    # Day-of-week (0..6) → sin/cos
    dow = idx.dayofweek.to_numpy()
    data_combined["sin_dow"] = np.sin(2 * np.pi * dow / 7.0).astype("float32")
    data_combined["cos_dow"] = np.cos(2 * np.pi * dow / 7.0).astype("float32")

    # Day-of-year (handles leap years) → sin/cos
    doy0 = (idx.dayofyear.to_numpy() - 1).astype("float32")  # 0..364/365
    days_in_year = np.where(idx.is_leap_year, 366.0, 365.0).astype("float32")
    doy_frac = (doy0 + tod_frac.astype("float32")) / days_in_year
    data_combined["sin_doy"] = np.sin(2 * np.pi * doy_frac).astype("float32")
    data_combined["cos_doy"] = np.cos(2 * np.pi * doy_frac).astype("float32")

    # Note: DST flags are only meaningful in local time. If your index is UTC, this tz_convert is correct.
    idx_local = idx
    if idx_local.tz is None:
        # Create tz-aware view for feature creation (does not change your stored index)
        idx_local = idx_local.tz_localize("Europe/Vienna", ambiguous="infer", nonexistent="shift_forward")
    else:
        idx_local = idx_local.tz_convert("Europe/Vienna")

    dst = idx_local.map(lambda ts: ts.dst())
    data_combined["is_dst"] = (dst != pd.Timedelta(0)).astype("int8")
    
    dst_s = pd.Series(dst, index=idx_local)

    # mark days with a DST transition (23h / 25h days)
    dst_change = dst_s != dst_s.shift(1)
    day_key = idx_local.normalize()

    data_combined["is_dst_transition_day"] = (
        pd.Series(dst_change, index=idx_local)
        .groupby(day_key)
        .transform("max")
        .astype("int8")
        .to_numpy()
    )

In [100]:
data_combined = data_combined.dropna()
print(len(data_combined))
data_combined

246241


,actual_load_mw,t2m,ssrd,tp,wind_speed,t2m_future,ssrd_future,tp_future,wind_speed_future,is_weekday,is_holiday,sin_tod,cos_tod,sin_dow,cos_dow,sin_doy,cos_doy,is_dst,is_dst_transition_day
period,,,,,,,,,,,,,,,,,,,
2018-10-01 00:00:00+00:00,5256.0,279.426453,0.0,3.671894e-10,0.693914,277.421722,0.0,0.000000e+00,1.758537,True,0,0.000000,1.000000,0.000000,1.000000,-0.999917,-0.012910,1,1
2018-10-01 00:15:00+00:00,5204.0,279.426453,0.0,3.671894e-10,0.693914,277.421722,0.0,0.000000e+00,1.758537,True,0,0.065403,0.997859,0.000000,1.000000,-0.999919,-0.012731,1,1
2018-10-01 00:30:00+00:00,5178.0,279.426453,0.0,3.671894e-10,0.693914,277.421722,0.0,0.000000e+00,1.758537,True,0,0.130526,0.991445,0.000000,1.000000,-0.999921,-0.012551,1,1
2018-10-01 00:45:00+00:00,5151.0,279.426453,0.0,3.671894e-10,0.693914,277.421722,0.0,0.000000e+00,1.758537,True,0,0.195090,0.980785,0.000000,1.000000,-0.999923,-0.012372,1,1
2018-10-01 01:00:00+00:00,5187.0,279.108917,0.0,3.959738e-08,0.583597,277.252350,0.0,6.201566e-08,1.709355,True,0,0.258819,0.965926,0.000000,1.000000,-0.999926,-0.012193,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-08 23:00:00+00:00,5667.0,281.157654,0.0,1.751735e-08,0.708287,281.028931,0.0,8.888660e-09,1.151026,True,0,-0.258819,0.965926,0.974928,-0.222521,-0.992311,0.123768,1,0
2025-10-08 23:15:00+00:00,5618.0,281.157654,0.0,1.751735e-08,0.708287,281.028931,0.0,8.888660e-09,1.151026,True,0,-0.195090,0.980785,0.974928,-0.222521,-0.992289,0.123946,1,0
2025-10-08 23:30:00+00:00,5534.0,281.157654,0.0,1.751735e-08,0.708287,281.028931,0.0,8.888660e-09,1.151026,True,0,-0.130526,0.991445,0.974928,-0.222521,-0.992267,0.124124,1,0


In [103]:
data_combined.to_parquet(OUTPUT_PATH / "data_combined.parquet")